## SQL

**SQL** (Structured Query Language) is a programming language that has operations to define, logically organize, manipulate, and perform calculations on data stored in a relational database management system (RDBMS).

SQL is a declarative language. This means that the user only needs to specify *what* kind of data they want, not *how* to obtain it. An example is shown below, with an imperative example for comparison:

- **Declarative**: Compute the table with columns "x" and "y" from table "A" where the values in "y" are greater than 100.00.
- **Imperative**: For each record in table "A", check if the record contains a value of "y" greater than 100. If so, then store the record's "x" and "y" attributes in a new table. Return the new table.

In this chapter, we will write SQL queries as Julia strings, then use the `SQLite.jl` package to execute the SQL query and read the result into a `DataFrame`.

### Executing SQL Queries through `SQLite`

To execute SQL queries from Julia, we will connect to a database using the [SQLite.jl](https://github.com/JuliaDatabases/SQLite.jl) library. Then we use the pipe operator `|>` to visualize the results in a `DataFrame`.

In [3]:
using SQLite
using DataFrames

db = SQLite.DB("sql_basics.db")

SQLite.DB("sql_basics.db")

This database contains one relation: `prices`. To display the relation we run a SQL query. Calling `SQLite.query` will execute the SQL query on the RDBMS. We then pipe the results with the `|>` operator into a DataFrame.

In [4]:
sql_expr = """
SELECT * 
FROM prices
"""

SQLite.Query(db, sql_expr) |> DataFrame

,retailer,product,price
,String⍰,String⍰,Float64⍰
1,Best Buy,Galaxy S9,719.0
2,Best Buy,iPod,200.0
3,Amazon,iPad,450.0
4,Amazon,Battery pack,24.87
5,Amazon,Chromebook,249.99
6,Target,iPod,215.0
7,Target,Surface Pro,799.0
8,Target,Google Pixel 2,659.0
9,Walmart,Chromebook,238.79


In [5]:
# HIDDEN
prices = DataFrame(
    retailer = ["Best Buy", "Best Buy", "Amazon", "Amazon", "Amazon", "Target", "Target", "Target", "Wallmart"],
    product = ["Galaxy S9", "iPod", "iPad", "Battery pack", "Chromebook", "iPod", "Surface Pro", "Google Pixel 2", "Chromebook"],
    price = [719.00, 200.00, 450.00, 24.87, 249.99, 215.00, 799.00, 659.00, 238.79]
);

Later in this section we will compare SQL queries with `DataFrames` method calls so we've created an identical DataFrame:

In [6]:
prices

,retailer,product,price
,String,String,Float64
1,Best Buy,Galaxy S9,719.0
2,Best Buy,iPod,200.0
3,Amazon,iPad,450.0
4,Amazon,Battery pack,24.87
5,Amazon,Chromebook,249.99
6,Target,iPod,215.0
7,Target,Surface Pro,799.0
8,Target,Google Pixel 2,659.0
9,Wallmart,Chromebook,238.79


## SQL Syntax

All SQL queries take the general form below:
```SQL
SELECT [DISTINCT] <column expression list>
FROM <relation>
[WHERE <predicate>]
[GROUP BY <column list>]
[HAVING <predicate>]
[ORDER BY <column list>]
[LIMIT <number>]
```

Note that:

1. **Everything in \[square brackets\] is optional.** A valid SQL query only needs a `SELECT` and a `FROM` statement.
2. **SQL SYNTAX IS GENERALLY WRITTEN IN CAPITAL LETTERS.** Although capitalization isn't required, it is common practice to write SQL syntax in capital letters. It also helps to visually structure your query for others to read.
3. `FROM` query blocks can reference one or more tables, although in this section we will only look at one table at a time for simplicity.

### SELECT and FROM

The two mandatory statements in a SQL query are:

* `SELECT` indicates the columns that we want to view.
* `FROM` indicates the tables from which we are selecting these columns.

To display the entire `prices` table, we run:

In [7]:
sql_expr = """
SELECT * 
FROM prices
"""

SQLite.Query(db, sql_expr) |> DataFrame

,retailer,product,price
,String⍰,String⍰,Float64⍰
1,Best Buy,Galaxy S9,719.0
2,Best Buy,iPod,200.0
3,Amazon,iPad,450.0
4,Amazon,Battery pack,24.87
5,Amazon,Chromebook,249.99
6,Target,iPod,215.0
7,Target,Surface Pro,799.0
8,Target,Google Pixel 2,659.0
9,Walmart,Chromebook,238.79


`SELECT *` returns every column in the original relation. To display only the retailers that are represented in `prices`, we add the `retailer` column to the `SELECT` statement.

In [8]:
sql_expr = """
SELECT retailer
FROM prices
"""

SQLite.Query(db, sql_expr) |> DataFrame

,retailer
,String⍰
1,Best Buy
2,Best Buy
3,Amazon
4,Amazon
5,Amazon
6,Target
7,Target
8,Target
9,Walmart


If we want a list of unique retailers, we can call the `DISTINCT` function to omit repeated values.

In [9]:
sql_expr = """
SELECT DISTINCT(retailer)
FROM prices
"""

SQLite.Query(db, sql_expr) |> DataFrame

,retailer
,String⍰
1,Best Buy
2,Amazon
3,Target
4,Walmart


This would be the functional equivalent of the following `DataFrames` code:

In [10]:
unique(prices.retailer)

4-element Array{String,1}:
 "Best Buy"
 "Amazon"  
 "Target"  
 "Wallmart"

Each RDBMS comes with its own set of functions that can be applied to attributes in the `SELECT` list, such as comparison operators, mathematical functions and operators, and string functions and operators. In Data 100 we use PostgreSQL, a mature RDBMS that comes with hundreds of such functions. The complete list is available [here](https://www.postgresql.org/docs/9.2/static/functions.html). Keep in mind that each RDBMS has a different set of functions for use in `SELECT`.

The following code converts all retailer names to uppercase and halves the product prices.

In [11]:
sql_expr = """
SELECT
    UPPER(retailer) AS retailer_caps,
    product,
    price / 2 AS half_price
FROM prices
"""

SQLite.Query(db, sql_expr) |> DataFrame

,retailer_caps,product,half_price
,String⍰,String⍰,Float64⍰
1,BEST BUY,Galaxy S9,359.5
2,BEST BUY,iPod,100.0
3,AMAZON,iPad,225.0
4,AMAZON,Battery pack,12.435
5,AMAZON,Chromebook,124.995
6,TARGET,iPod,107.5
7,TARGET,Surface Pro,399.5
8,TARGET,Google Pixel 2,329.5
9,WALMART,Chromebook,119.395


Notice that we can **alias** the columns (assign another name) with `AS` so that the columns appear with this new name in the output table. This does not modify the names of the columns in the source relation.

### WHERE

The `WHERE` clause allows us to specify certain constraints for the returned data; these constraints are often referred to as **predicates**. For example, to retrieve only gadgets that are under $500:

In [12]:
sql_expr = """
SELECT *
FROM prices
WHERE price < 500
"""

SQLite.Query(db, sql_expr) |> DataFrame

,retailer,product,price
,String⍰,String⍰,Float64⍰
1,Best Buy,iPod,200.0
2,Amazon,iPad,450.0
3,Amazon,Battery pack,24.87
4,Amazon,Chromebook,249.99
5,Target,iPod,215.0
6,Walmart,Chromebook,238.79


We can also use the operators `AND`, `OR`, and `NOT` to further constrain our SQL query. To find an item on Amazon without a battery pack under $300, we write:

In [13]:
sql_expr = """
SELECT *
FROM prices
WHERE retailer = 'Amazon'
    AND NOT product = 'Battery pack'
    AND price < 300
"""

SQLite.Query(db, sql_expr) |> DataFrame

,retailer,product,price
,String⍰,String⍰,Float64⍰
1,Amazon,Chromebook,249.99


The equivalent operation in `DataFrames` is:

In [15]:
prices[(prices.retailer .== "Amazon") .& (prices.product .!== "Battery pack") .& (prices.price .< 300), :]

,retailer,product,price
,String,String,Float64
1,Amazon,Chromebook,249.99


### Aggregate Functions

So far, we've only worked with data from the existing rows in the table; that is, all of our returned tables have been some subset of the entries found in the table. But to conduct data analysis, we'll want to compute aggregate values over our data. In SQL, these are called **aggregate functions**. 

If we want to find the average price of all gadgets in the `prices` relation:

In [16]:
sql_expr = """
SELECT AVG(price) AS avg_price
FROM prices
"""

SQLite.Query(db, sql_expr) |> DataFrame

,avg_price
,Float64⍰
1,395.072


Equivalently, in Julia:

In [18]:
using Statistics

mean(prices.price)

395.0722222222222

A complete list of PostgreSQL aggregate functions can be found [here](https://www.postgresql.org/docs/9.2/static/functions.html). Though we're using PostgreSQL as our primary version of SQL, keep in mind that there are many other variations of SQL (MySQL, SQLite, etc.) that use different function names and have different functions available.

### GROUP BY and HAVING

With aggregate functions, we can execute more complicated SQL queries. To operate on more granular aggregate data, we can use the following two clauses:
- `GROUP BY` takes a list of columns and groups the table. Similar to the functions [groupby](http://juliadata.github.io/DataFrames.jl/v0.17.0/lib/functions.html#DataFrames.groupby) and [by](http://juliadata.github.io/DataFrames.jl/v0.17.0/lib/functions.html#DataFrames.by) from `DataFrames`.
- `HAVING` is functionally similar to `WHERE`, but is used exclusively to apply predicates to aggregated data. (Note that in order to use `HAVING`, it must be preceded by a `GROUP BY` clause.)

**Important**: When using `GROUP BY`, all columns in the `SELECT` clause must be either listed in the `GROUP BY` clause or have an aggregate function applied to them.

We can use these statements to find the maximum price at each retailer.

In [19]:
sql_expr = """
SELECT retailer, MAX(price) as max_price
FROM prices
GROUP BY retailer
"""

SQLite.Query(db, sql_expr) |> DataFrame

,retailer,max_price
,String⍰,Float64⍰
1,Amazon,450.0
2,Best Buy,719.0
3,Target,799.0
4,Walmart,238.79


Let's say we have a client with expensive taste and only want to find retailers that sell gadgets over $700. Note that we must use `HAVING` to define predicates on aggregated columns; we can't use `WHERE` to filter an aggregated column. To compute a list of retailers and accompanying prices that satisfy our needs, we run:

In [20]:
sql_expr = """
SELECT retailer, MAX(price) as max_price
FROM prices
GROUP BY retailer
HAVING max_price > 700
"""

SQLite.Query(db, sql_expr) |> DataFrame

,retailer,max_price
,String⍰,Float64⍰
1,Best Buy,719.0
2,Target,799.0


For comparison, we recreate the same table in `DataFrames`:

In [22]:
max_prices = by(prices, :retailer, :price => maximum)
max_prices[max_prices.price_maximum .> 700, :]

,retailer,price_maximum
,String,Float64
1,Best Buy,719.0
2,Target,799.0


### ORDER BY and LIMIT

These clauses allow us to control the presentation of the data:
- `ORDER BY` lets us present the data in lexicographic order of column values. By default, ORDER BY uses ascending order (`ASC`) but we can specify descending order using `DESC`.
- `LIMIT` controls how many tuples are displayed.

Let's display the three cheapest items in our `prices` table:

In [23]:
sql_expr = """
SELECT *
FROM prices
ORDER BY price ASC
LIMIT 3
"""

SQLite.Query(db, sql_expr) |> DataFrame

,retailer,product,price
,String⍰,String⍰,Float64⍰
1,Amazon,Battery pack,24.87
2,Best Buy,iPod,200.0
3,Target,iPod,215.0


Note that we didn't have to include the `ASC` keyword since `ORDER BY` returns data in ascending order by default.
For comparison, in `DataFrames`:

In [24]:
first(sort(prices, :price), 3)

,retailer,product,price
,String,String,Float64
1,Amazon,Battery pack,24.87
2,Best Buy,iPod,200.0
3,Target,iPod,215.0


### Conceptual SQL Evaluation

Clauses in a SQL query are executed in a specific order. Unfortunately, this order differs from the order that the clauses are written in a SQL query. From first executed to last:

1. `FROM`: One or more source tables
2. `WHERE`: Apply selection qualifications (eliminate rows)
3. `GROUP BY`: Form groups and aggregate
4. `HAVING`: Eliminate groups
5. `SELECT`: Select columns

**Note on `WHERE` vs. `HAVING`**: Since the `WHERE` clause is processed before applying `GROUP BY`, the `WHERE` clause cannot make use of aggregated values. To define predicates based on aggregated values, we must use the `HAVING` clause.

## Summary

We have introduced SQL syntax and the most important SQL statements needed to conduct data analysis using a relational database management system.